In [1]:
import torch as th
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import plot

In [2]:
device = th.device('cpu')

In [3]:
def tile_them(array_list, shape, reshape):
  return [np.tile(array, shape).view(reshape) for array in array_list]

In [4]:
P = 10
R = 30
Q = 100
N = 11
M = 20
W1 = 6
W2 = 7
W3 = 8
W4 = 9
X = 40

METER = 1
SEC = 1
LAMBDA = 1 * METER
K = 2 * np.pi / LAMBDA

dtype = th.complex64

width1 = np.linspace(0 * METER, 1 * METER, W1)
width2 = np.linspace(0 * METER, 1 * METER, W2)
width3 = np.linspace(0 * METER, 1 * METER, W3)
width4 = np.linspace(0 * METER, 1 * METER, W4)


x1 = np.linspace(0 * METER, 10 * METER, X)
x2 = np.linspace(0 * METER, 10 * METER, X)


position1 = np.linspace(0 * METER, 10 * METER, N)
position2 = np.linspace(0 * METER, 10 * METER, N)
position3 = np.linspace(0 * METER, 15 * METER, M)
position4 = np.linspace(0 * METER, 15 * METER, N)

In [5]:
field1 = th.tensor(np.linspace(np.add.outer(position1, -width1 / 2), np.add.outer(position1, width1 / 2), P), dtype=dtype, device=device).permute(2, 0, 1)
field2 = th.tensor(np.linspace(np.add.outer(position2, -width2 / 2), np.add.outer(position2, width2 / 2), P), dtype=dtype, device=device).permute(2, 0, 1)
field3 = th.tensor(np.linspace(np.add.outer(position3, -width3 / 2), np.add.outer(position3, width3 / 2), Q), dtype=dtype, device=device).permute(2, 0, 1)
field4 = th.tensor(np.linspace(np.add.outer(position4, -width4 / 2), np.add.outer(position4, width4 / 2), P), dtype=dtype, device=device).permute(2, 0, 1)
field1.shape

torch.Size([6, 10, 11])

In [29]:
E1 = th.ones_like(field1, dtype=dtype, device=device)
E2 = th.ones_like(field2, dtype=dtype, device=device)
R1 = th.tile(field1, (W2, W3, Q, M)).view(W2, W3, W1, Q, P, M, N).permute(2, 0, 1, 3, 4, 5, 6)
R2 = th.tile(field2, (W1, W3, Q, M)).view(W1, W3, W2, Q, P, M, N).permute(0, 2, 1, 3, 4, 5, 6)
R3 = th.tile(field3, (W1, W2, P, N)).view(W1, W2, W3, P, Q, N, M).permute(0, 1, 2, 4, 3, 6, 5)


torch.Size([6, 7, 8, 100, 10, 20, 11])

In [30]:
D1 = R1 - R3
D2 = R2 - R3
D1 = th.sqrt(x1 ** 2 + D1 ** 2)
D2 = th.sqrt(x1 ** 2 + D2 ** 2)
E3 = th.einsum("qpmn,pn->pqm", th.exp(1j * D1 * K) / D1, E1) + th.einsum("qpmn,pn->pqm", th.exp(1j * D2 * K) / D2, E2)


TypeError: Concatenation operation is not implemented for NumPy arrays, use np.concatenate() instead. Please do not rely on this error; it may not be given on all Python implementations.

In [ ]:

R3 = th.tile(field3, (R, N)).view(R, Q, N, M)
R4 = th.tile(field4, (Q, M)).view(Q, R, M, N).permute((1, 0, 3, 2))

D3 = R3 - R4
D3 = th.sqrt(x2 ** 2 + D3 ** 2)
E4 = th.einsum("rqmn,pqn->pqrm", th.exp(1j * D3 * K) / D3, E3)
result = th.abs(E4)
plot(result[0, 0, :, 0].to("cpu").numpy())

In [ ]:
R2 = th.tile(field2, (Q, M)).view(Q, P, M, N)
R3 = th.tile(field3, (P, N)).view(P, Q, N, M).permute((1, 0, 3, 2))

D1 = R1 - R3
D2 = R2 - R3
D1 = th.sqrt(x1 ** 2 + D1 ** 2)
D2 = th.sqrt(x1 ** 2 + D2 ** 2)
E3 = th.einsum("qpmn,pn->pqm", th.exp(1j * D1 * K) / D1, E1) + th.einsum("qpmn,pn->pqm", th.exp(1j * D2 * K) / D2, E2)

R3 = th.tile(field3, (R, N)).view(R, Q, N, M)
R4 = th.tile(field4, (Q, M)).view(Q, R, M, N).permute((1, 0, 3, 2))

D3 = R3 - R4
D3 = th.sqrt(x2 ** 2 + D3 ** 2)
E4 = th.einsum("rqmn,pqn->pqrm", th.exp(1j * D3 * K) / D3, E3)
result = th.abs(E4)
plot(result[0, 0, :, 0].to("cpu").numpy())